# 多层感知机 --- 使用Gluon

我们只需要稍微改动[多类Logistic回归](http://zh.gluon.ai/chapter_supervised-learning/softmax-regression-gluon.html)来实现多层感知机。

## 定义模型

唯一的区别在这里，我们加了一行进来。

In [8]:
from mxnet import gluon
from mxnet import autograd, nd

def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def evaluate_accuracy(data_iterator, net):
    acc = 0.
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(256, activation="relu"))
    net.add(gluon.nn.Dense(10))
net.initialize()

## 读取数据并训练

In [2]:
batch_size = 256
root = 'E:/Data/MXNet/fashion_mnist'

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform, root= root)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform, root= root)

train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

C:\Anaconda3\lib\site-packages\mxnet\gluon\data\vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
C:\Anaconda3\lib\site-packages\mxnet\gluon\data\vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [10]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})

for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.342235, Train acc 0.874546, Test acc 0.882617
Epoch 1. Loss: 0.329278, Train acc 0.879006, Test acc 0.880566
Epoch 2. Loss: 0.318365, Train acc 0.881200, Test acc 0.887793
Epoch 3. Loss: 0.305880, Train acc 0.888708, Test acc 0.890625
Epoch 4. Loss: 0.300178, Train acc 0.888863, Test acc 0.881348
Epoch 5. Loss: 0.287381, Train acc 0.893168, Test acc 0.891602
Epoch 6. Loss: 0.282453, Train acc 0.893800, Test acc 0.887988
Epoch 7. Loss: 0.274919, Train acc 0.897634, Test acc 0.890820
Epoch 8. Loss: 0.269042, Train acc 0.900493, Test acc 0.886914
Epoch 9. Loss: 0.261999, Train acc 0.902837, Test acc 0.895703
